# Projeto 3 de Ciência dos Dados

## Integrantes

- Bruno Domingues
- Stefano Moretti
- Thomas Queiroz

### Introdução

O projeto consiste na montagem de um classificador que tem como input uma foto de uma pessoa qualquer, e com base na foto retornar o sexo biológico da pessoa na foto.

Para tal, usamos um dataset composto de 13000 fotos, chamado de "Labeled Faces in the Wild". Nele, há 13000 fotos de 5749 pessoas diferentes, e dentre estas, 1680 pessoas possuem mais de uma foto. A única "restrição" presente é que as fotos foram obtidas através do detector de faces Viola-Jones. Há quatro versões do dataset que podem ser encontradas:
- versão original
- versão com alinhamento por deep funneling
- versão com alinhamento por LFW-a
- versão com alinhamento por funneling

No caso, optamos pela última, uma vez que acreditamos que o alinhamento das faces torna o processo de detectar padrões mais fácil.

Com o dataset em mãos, começamos a adaptá-lo para nosso projeto.

### Limpeza dos dados

Primeiro, obtivemos o local onde as fotos se encontravam no computador. Em seguida, unimos as fotos com 2 arquivos txt em uma pasta, sendo que estes arquivos continham os nomes de homens e mulheres presentes no dataset, gerando nossos labels. Após isto, começamos a editar as fotos: reduzimos o tamanho da foto de 250x250 para 125x125, depois centralizamos melhor a imagem e reduzimos para 80x80; em seguida, reduzimos para 40x40 pixels. Por fim, transformamos as fotos em listas unidimensionais em que cada pixel representa um elemento, de forma que o classificador possa analisar cada pixel individualmente.

In [1]:
import cv2, os
import numpy as np
from time import time

In [2]:
path_ori = os.getcwd()
path_dataset = "C:\\Users\\thoma\\Documents\\INSPER\\2 Sem\\Ciência dos Dados\\lfw_funneled"
# path_dataset = "C:\\Users\\Bruno\\Documents\\Insper\\2º Semestre\\Ciência dos Dados\\Projeto_3\\lfw_funneled"
# path_dataset = "C:\\Users\\Stefano Moretti\\Documents\\INSPER\\2 Sem\\Ciência dos Dados\\lfw_funneled"

In [3]:
initialloadTime = time()

os.chdir(path_dataset)

with open('male_names.txt','r') as m:
    male = m.readlines()
    male = [i.replace("\n","") for i in male]

with open('female_names.txt','r') as fm:
    female = fm.readlines()
    female = [i.replace("\n","") for i in female]


m = []
fm = []

for pasta in os.listdir(path_dataset):
    if "." not in pasta:
        path_folder = ".\\" + pasta
        for im in os.listdir(path_folder):
            tempIm = cv2.imread(path_folder + "\\" + im) # 250 x 250

            tempIm = cv2.pyrDown(tempIm) # 125 x 125
            tempIm = tempIm[20:100, 20:100] # 80 x 80
            tempIm = cv2.pyrDown(tempIm) # 40 x 40
            tempIm = cv2.cvtColor(tempIm, cv2.COLOR_BGR2RGB)
            
            tempIm = tempIm.flatten()
            
            if im in male:
                m.append(tempIm)
            elif im in female:
                fm.append(tempIm)

os.chdir(path_ori)

finalloadTime = time()
f'Tempo para carregar as fotos: {finalloadTime - initialloadTime :.2f} segundos'

'Tempo para carregar as fotos: 48.31 segundos'

Após a limpeza dos dados, juntamos as fotos de homens e mulheres, e embaralhamos as fotos. Em seguida, separamos o dataset em 2 partes: 70% das fotos seriam usadas para o treino do classificador e o restante seria usado para teste.

In [4]:
X = m + fm
y = [0]*len(m) + [1]*len(fm)

c = list(zip(X, y))

np.random.shuffle(c)

X, y = zip(*c)

In [7]:
index = round(len(X) * 0.7)
X_train, X_test = X[:index], X[index:]
y_train, y_test = y[:index], y[index:]

Com o dataset de treino e teste separado, pudemos seguir para a escolha de nosso classificador.

### Classificador

Após testar diversos classificadores que podem ser encontrados na biblioteca do scikit learn, optamos por usar o Gradient Boosting Classifier, uma vez que ele nos retornou a melhor acurácia.

In [10]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
import pickle

train_model = True

if train_model:
    comeco_treino = time()
    clf = GradientBoostingClassifier()
    clf.fit(X_train, y_train)

    fim_treino = time()
    print(f'Tempo para treinar o modelo: {fim_treino - comeco_treino:.2f} segundos')

    comeco_prev = time()
    y_pred = clf.predict(X_test)

    fim_prev = time()
    print(f'Tempo para testar o modelo: {fim_prev - comeco_prev :.2f} segundos')


    print('Acurácia: {0:.2f} %'.format(100 * accuracy_score(y_test, y_pred)))
    
    # Salvando o modelo
    pickle.dump(clf, open('model.p', 'wb'))
else:
    # Carregando o modelo
    clf = pickle.load(open('model.p', 'rb'))
    
    comeco_prev = time()
    y_pred = clf.predict(X_test)

    fim_prev = time()
    print(f'Tempo para testar o modelo: {fim_prev - comeco_prev :.2f} segundos')


    print('Acurácia: {0:.2f} %'.format(100 * accuracy_score(y_test, y_pred)))

Tempo para treinar o modelo: 467.36 segundos
Tempo para testar o modelo: 0.13 segundos
Acurácia: 87.53 %


Mas afinal, o que o Gradient Boosting faz?

O Gradient Boosting funciona através do acréscimo sequencial de preditores em um conjunto, de modo que cada um corrija seu antecessor. Este método tenta adaptar o novo preditor para prever os *erros residuais* feitos pelo preditor anterior. Um exemplo disto seria usando árvores de decisão como preditores: imagine que usamos uma árvore que faz um fit do $X_{train}$ e o $y_{train}$. Em seguida, usamos uma segunda árvore de decisão, de modo que ela faça um fit do $X_{train}$ e do erro residual da árvore anterior (dado por $\hat{y_2}$). Após isso, usamos uma terceira árvore de decisão, que usa ainda o $X_{train}$ e o erro residual da segunda árvore. Com isso, temos um conjunto de três árvores, de modo que a previsão deste conjunto é dada pela soma das previsões de cada árvore. Para decidir o número de árvores mais adequado, pode ser implementado o que é chamado de early stopping, mas optamos por não utilizar uma vez que exigia muito tempo para treinar o modelo.

### Bibliografia

Gary B. Huang, Manu Ramesh, Tamara Berg, and Erik Learned-Miller.
**Labeled Faces in the Wild: A Database for Studying Face Recognition in Unconstrained Environments.**
University of Massachusetts, Amherst, Technical Report 07-49, October, 2007.
http://vis-www.cs.umass.edu/lfw/

Aurelien Geron.
**Hands–On Machine Learning with Scikit–Learn and TensorFlow**

Slides de aula

https://github.com/Insper/CD18